In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd

In [2]:
import logging
import pyspark
from delta import *
import json

# Load the configuration JSON file
with open('/usr/local/spark/conf/spark-defaults.json', 'r') as f:
    config = json.load(f)

# Initialize the Spark session builder
builder = pyspark.sql.SparkSession.builder.appName("MyApp")

# Read the packages from the text file
packages = []
with open('/usr/local/spark/conf/packages.txt', 'r') as file:
    # Read each line and strip newlines or extra spaces
    packages = [line.strip() for line in file if line.strip()]

# Add packages to the Spark session configuration
builder.config("spark.jars.packages", ",".join(packages))

# Apply the configurations from the JSON file to the Spark session
for key, value in config.items():
    builder.config(key, value)

# Configure Spark with Delta Lake (if needed)
spark = configure_spark_with_delta_pip(builder).getOrCreate()


# Now you can use the Spark session
spark

In [6]:
trgt_path_processed = "/mnt/Calendar/Calendar_Parquet/"
trgt_path_csv = "/mnt/Calendar/Calendar_Processed/"

In [7]:
# Create a DataFrame with date range
start_date = "2000-01-01"
end_date = "2050-12-31"

In [8]:
# Create a DataFrame with a single row containing the start and end date
date_range_df = spark.createDataFrame([(start_date, end_date)], ["start_date", "end_date"])

# Generate date sequence
date_sequence_df = date_range_df.select(
    sequence(
        to_date(date_range_df.start_date).alias("start_date"),
        to_date(date_range_df.end_date).alias("end_date")
    ).alias("date")
)

In [9]:
# Explode the sequence into separate rows
df_date = date_sequence_df.selectExpr("explode(date) as date")

In [10]:
df_output = df_date.withColumn("DateSK", regexp_replace("date", "-", "")).withColumn("Year", year("date"))\
    .withColumn("Month",date_format("date","MMMM")).withColumn("Quarter",concat(year("date"), lit(" Q"), quarter("date")))

In [11]:
print(df_output.count())

18628


In [12]:
df_output.createOrReplaceTempView("vw_source")

In [13]:
column_name = df_output.columns
set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
query = f"""MERGE INTO delta.`{trgt_path_processed}` AS target USING vw_source AS source ON target.DateSK = source.DateSK WHEN MATCHED THEN UPDATE SET {set_clause}"""
print(query)

MERGE INTO delta.`/mnt/Calendar/Calendar_Parquet/` AS target USING vw_source AS source ON target.DateSK = source.DateSK WHEN MATCHED THEN UPDATE SET target.date = source.date, target.DateSK = source.DateSK, target.Year = source.Year, target.Month = source.Month, target.Quarter = source.Quarter


In [16]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    spark.sql(query)
else :
    df_output.write.format("delta").save(trgt_path_processed)
    # spark.sql(f"""CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source""")

Py4JJavaError: An error occurred while calling o131.save.
: com.google.common.util.concurrent.ExecutionError: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.delta.storage.DelegatingLogStore$
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2261)
	at com.google.common.cache.LocalCache.get(LocalCache.java:4000)
	at com.google.common.cache.LocalCache$LocalManualCache.get(LocalCache.java:4789)
	at org.apache.spark.sql.delta.DeltaLog$.getDeltaLogFromCache$1(DeltaLog.scala:801)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:811)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:643)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:172)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.

In [14]:
df_output.write.format("delta").mode("overwrite").save(trgt_path_processed)

In [15]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1).write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)

In [16]:
spark.read.format("delta").load(trgt_path_processed).count()